# Introducción al Downscaling Climático

El **downscaling** o escalamiento climático es una técnica que permite traducir datos de modelos climáticos o reanálisis, de baja resolución espacial, a escalas locales o puntuales, con mayor detalle. Este proceso es esencial para aplicaciones hidrológicas, agrícolas y de gestión de riesgos climáticos.

## Métodos comunes de downscaling:

1. **Tasa Altitudinal (Lapse Rate Correction)**  
   Ajusta las temperaturas simuladas en función de la diferencia de altitud entre el punto de reanálisis y la estación meteorológica, utilizando una tasa estándar (por ejemplo, -6.5 °C/km).  
   - Simple, pero útil en regiones montañosas.
   - No considera otros factores meteorológicos.

2. **Regresión Lineal**  
   Utiliza una relación estadística entre variables del reanálisis (predictoras) y datos observados (respuesta), generalmente con una única variable independiente.  
   - Método interpretable.
   - Supone una relación lineal fija entre variables.

3. **XGBoost (Extreme Gradient Boosting)**  
   Técnica de **machine learning** basada en árboles de decisión optimizados por gradiente. Permite capturar relaciones no lineales y multivariadas entre datos de reanálisis y observaciones locales.  
   - Alto rendimiento predictivo.
   - Requiere más datos y calibración cuidadosa.

Estas técnicas pueden aplicarse a variables como temperatura, precipitación, viento o radiación, utilizando datos como los del reanálisis **ERA5** y observaciones de estaciones meteorológicas.


## Importar librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import xarray as xr
from sklearn.linear_model import LinearRegression
import math
from scipy.stats import pearsonr
# Importa la función de densidad de probabilidad kernel gaussiana de scipy
from scipy.stats import gaussian_kde


## Leer datos

### Datos ERA5

In [ ]:
path_dir = '../data/ERA5_AG/'

ds1 = xr.open_dataset(f'{path_dir}ERA5_{2016}_ins.nc')
ds2 = xr.open_dataset(f'{path_dir}ERA5_{2016}_accum.nc')
ds3 = xr.open_dataset(f'{path_dir}ERA5_{2016}_rrr.nc')

In [ ]:
ds3_3h = ds3.resample(valid_time='3h').sum()
ds3_3h

In [ ]:
ds = xr.merge([ds1, ds2, ds3_3h])
ds

In [ ]:
years = np.arange(2016, 2019, 1)
print(years)

In [ ]:
ds_list = []
for year in years:
    ds1 = xr.open_dataset(f'{path_dir}ERA5_{year}_ins.nc')
    ds2 = xr.open_dataset(f'{path_dir}ERA5_{year}_accum.nc')
    ds3 = xr.open_dataset(f'{path_dir}ERA5_{year}_rrr.nc').resample(valid_time='3h').sum()
    ds_list.append(xr.merge([ds1, ds2, ds3]))


In [ ]:
ds_list

In [ ]:
ds = xr.merge(ds_list)
ds

In [ ]:
df_era5 = ds.isel(latitude=0, longitude=0).to_dataframe().drop(columns=['number', 'latitude', 'longitude', 'expver'])
df_era5

In [ ]:
## Velocidad del viento
U10 = np.sqrt(df_era5['u10']**2 + df_era5['v10']**2)
df_era5['ws2'] = U10 * (np.log(2 / (2.12 * 1000)) / np.log(10 / (2.12 * 1000)))
df_era5

In [ ]:
## Humedad relativa

# Constants for humidity calculation
T0 = 273.16
a1 = 611.21
a3 = 17.502
a4 = 32.19
R_dry = 287.0597
R_vap = 461.5250


T_e_sat = a1 * np.exp(a3 * ((df_era5['t2m'] - T0) / (df_era5['t2m'] - a4)))
Td_e_sat = a1 * np.exp(a3 * ((df_era5['d2m'] - T0) / (df_era5['d2m'] - a4)))
df_era5['rh']  = 100 * Td_e_sat / T_e_sat
df_era5['rh'] = df_era5['rh'].clip(lower=0, upper=100)
df_era5

In [ ]:
g = 9.81 ## m/s**2
df_era5['t2m'] = df_era5['t2m'] - 272.15
df_era5['sp'] = df_era5['sp'] / 100
df_era5['z'] = df_era5['z'] / g
df_era5['ssrd'] = df_era5['ssrd'] / 3600 # J a W
df_era5['strd'] = df_era5['strd'] / 3600 # J a W
df_era5['tp'] = df_era5['tp'] * 1000 # m a mm
df_era5


### Datos AWS

In [ ]:
filename = '../data/arteson_2016_2018_1h.csv'
df_aws   =  pd.read_csv(filename, sep='\t')
df_aws.TIMESTAMP = pd.to_datetime(df_aws.TIMESTAMP)
df_aws.set_index('TIMESTAMP', inplace=True)
vars_aws = df_aws.columns

df_aws['data_hora_LT'] = df_aws.index.tz_localize('America/Lima')
df_aws['data_hora_utc'] = df_aws['data_hora_LT'].dt.tz_convert('UTC')
# Remover o timezone (ex: -05:00)
df_aws['TIMESTAMP'] = df_aws['data_hora_utc'].dt.tz_localize(None)
df_aws.set_index('TIMESTAMP', inplace=True)
df_aws = df_aws[vars_aws]
df_aws['SWin_Avg'] = df_aws['SWin_Avg'].clip(lower=0, upper=1400)
df_aws

In [ ]:
agg_dict = {
    'SWin_Avg': 'mean',
    'Ta_Avg': 'mean',
    'Rh_Avg': 'mean',
    'WiSp': 'mean',
    'pressure': 'mean',
    'precip_Tot': 'sum'
}

df_aws = df_aws.resample('3h').agg(agg_dict)
df_aws['pressure'] = df_aws['pressure'] - 172 ## 
df_aws

In [ ]:
df_aws.to_csv('../data/aws_AG_3h.csv', sep='\t')

## Escalamiento ERA5 para AWS T2

In [ ]:
# Altura de la estación y del punto ERA5
alt_era5 = df_era5['z'].values[0]  # m
alt_estacion = 4817  # m
lapse_rate = -0.0065  # °C/m

print(alt_era5, alt_estacion)

In [ ]:
# Juntar ambos df
df_era5_aws = pd.merge(df_era5, df_aws, left_index=True, right_index=True, how='left')
df_era5_aws

In [ ]:
df_era5_aws.columns

### Ecalamiento por Tasa Altitudinal

In [ ]:
df_era5_aws['t2m_corr_lp'] = df_era5_aws['t2m'] + lapse_rate * (alt_estacion - alt_era5)
df_era5_aws

### Escalamiento por regresion lineal

In [ ]:
df_era5_aws['ele_aws'] = alt_estacion

In [ ]:
df_era5_aws

In [ ]:
# -------------------------------
# 1. Preparación de datos
# -------------------------------

# Definir las variables que se utilizarán en el análisis:
# - 't2m': temperatura del aire a 2 metros (ERA5)
# - 'ssrd': radiación solar superficial descendente (ERA5)
# - 'u10' y 'v10': componentes zonal y meridional del viento a 10 m (ERA5)
# - 'z': altitud del punto ERA5
# - 'ele_aws': altitud de la estación meteorológica
# - 'Ta_Avg': temperatura media observada en la estación meteorológica
#vars = ['t2m', 'ssrd', 'u10', 'v10', 'Ta_Avg']
vars = ['t2m', 'ssrd', 'u10', 'v10', 'z', 'ele_aws', 'Ta_Avg']

# Crear una copia del DataFrame original que contenga únicamente las variables seleccionadas y eliminar filas con valores nulos
df_era5_aws_nonan = df_era5_aws[vars].copy().dropna()

# Definir las variables predictoras (features) tomadas de ERA5 y altitud de la estación
#X = df_era5_aws_nonan[['t2m', 'ssrd', 'u10', 'v10']]
X = df_era5_aws_nonan[['t2m', 'ssrd', 'u10', 'v10', 'z', 'ele_aws']]

# Definir la variable objetivo: temperatura promedio observada (de la estación)
y = df_era5_aws_nonan['Ta_Avg']

# Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
# La semilla random_state=42 permite obtener los mismos resultados al ejecutar varias veces
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir el conjunto de prueba (features) para revisar su contenido
print(len(X_train), len(X_test))


In [ ]:
# -------------------------------
# 2. Modelo de regresión lineal
# -------------------------------

# Crear una instancia del modelo de regresión lineal de scikit-learn
modelo_lr = LinearRegression()

# Ajustar (entrenar) el modelo usando los datos de entrenamiento
modelo_lr.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_lr = modelo_lr.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los valores reales
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

# Calcular el coeficiente de determinación R² (explica qué proporción de la varianza se explica por el modelo)
r2_lr = r2_score(y_test, y_pred_lr)

# Imprimir el RMSE con dos decimales, indicando el error promedio de las predicciones
print(f"[Regresión Lineal] RMSE: {rmse_lr:.2f} °C")

# Imprimir el R² con dos decimales, indicando la calidad del ajuste del modelo
print(f"[Regresión Lineal] R²: {r2_lr:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['Ta_Avg'])
df_era5_aws['t2m_corr_lineal'] = modelo_lr.predict(X_full)
df_era5_aws

### Escalamiento con Machine Learing

In [ ]:
# -------------------------------
# 4. Modelo XGBoost
# -------------------------------

# Crear un modelo de regresión XGBoost con hiperparámetros definidos:
# - n_estimators=100: número de árboles a construir.
# - learning_rate=0.1: tasa de aprendizaje que controla la contribución de cada árbol.
# - max_depth=4: profundidad máxima de cada árbol (controla la complejidad del modelo).
# - random_state=42: semilla aleatoria para reproducibilidad.
modelo_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)

# Entrenar el modelo con los datos de entrenamiento (X_train, y_train)
modelo_xgb.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_xgb = modelo_xgb.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los datos reales
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

# Calcular el coeficiente de determinación R² (qué tan bien el modelo explica la variabilidad de los datos)
r2_xgb = r2_score(y_test, y_pred_xgb)

# Imprimir el RMSE formateado con dos decimales
print(f"[XGBoost] RMSE: {rmse_xgb:.2f} °C")

# Imprimir el R² formateado con dos decimales
print(f"[XGBoost] R²: {r2_xgb:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['Ta_Avg'])
df_era5_aws['t2m_corr_xgb'] = modelo_xgb.predict(X_full)
df_era5_aws

## Visualización

In [ ]:
vars_t2 = ['Ta_Avg', 't2m', 't2m_corr_lp', 't2m_corr_lineal', 't2m_corr_xgb']
df_t2 = df_era5_aws[vars_t2]
df_t2

In [ ]:

def plot_series(df, n_toplot=10**10,):


    # Crea una copia del DataFrame original y elimina filas con valores faltantes
    df_nan = df.copy().dropna()

    # Renombra las columnas a 'OBS' (observado) y 'SIM' (simulado)
    df_nan.columns = ['OBS', 'SIM']

    # Extrae los valores como arrays numpy
    y1 = df_nan['OBS'].values  # serie observada
    y2 = df_nan['SIM'].values  # serie simulada

    # Crea un arreglo de índices de la misma longitud que las series
    idxs = np.arange(len(y1))

    # Mezcla aleatoriamente los índices (baraja)
    np.random.shuffle(idxs)

    # Selecciona hasta 'n_toplot' puntos aleatorios de cada serie
    y_expected = y1.reshape(-1)[idxs[:n_toplot]]   # valores observados aleatorios
    y_predicted = y2.reshape(-1)[idxs[:n_toplot]]  # valores simulados aleatorios

    # Combina ambos arrays en una matriz 2xN para el cálculo de densidad
    xy = np.vstack([y_expected, y_predicted])

    # Calcula la densidad de puntos usando KDE (kernel density estimation)
    z = gaussian_kde(xy)(xy)  # devuelve una densidad para cada punto (x, y)

    # Ordena los puntos por densidad ascendente (para que los más densos queden encima al graficar)
    idx = z.argsort()

    # Reordena los datos según esa densidad
    y_plt, ann_plt, z = y_expected[idx], y_predicted[idx], z[idx]

    # Devuelve los datos listos para graficar con color basado en densidad
    return y_plt, ann_plt, z


def get_metrics(df):
    # Elimina filas con valores faltantes (NaNs)
    df = df.dropna()
    
    # Renombra las columnas a 'OBS' (observado) y 'SIM' (simulado)
    df.columns = ['OBS', 'SIM']
    
    # Calcula el coeficiente de correlación de Pearson entre OBS y SIM
    r = pearsonr(df['OBS'], df['SIM'])[0]
    
    # Calcula el error cuadrático medio (MSE)
    MSE = np.square(np.subtract(df['OBS'], df['SIM'])).mean()
    
    # Calcula el sesgo medio (MBE): diferencia media entre simulado y observado
    MBE = np.mean(df['SIM'] - df['OBS']) 
    
    # Calcula la raíz do MSE, o sea, el RMSE
    RMSE = math.sqrt(MSE)

    # Crea un string con los resultados formateados para mostrar en una figura o tabla
    textstr = '\n'.join((
        r'$r=%.2f$' % (r, ),         # Correlación
        r'$RMSE=%.3f$' % (RMSE, ),   # RMSE
        r'$Bias=%.3f$' % (MBE, ),))   # Sesgo
    
    # Retorna el texto con las métricas
    return textstr


Este código es útil para generar datos listos para un gráfico de dispersión (scatter) entre dos variables (observada vs simulada), coloreando los puntos según su densidad con KDE. Eso ayuda a visualizar grandes cantidades de datos sin que las zonas densas se oculten.

In [ ]:
# Lista de variables simuladas para comparar con observados
sim_vars = ['t2m', 't2m_corr_lp', 't2m_corr_lineal', 't2m_corr_xgb']

plt.figure(figsize=(16, 12))

for i, var in enumerate(sim_vars, 1):
    # Preparar DataFrame con las columnas observadas y simuladas para la función plot_series y get_metrics
    df_plot = df_t2[['Ta_Avg', var]].copy()
    
    # Obtener datos para graficar con densidad KDE
    y_obs, y_sim, dens = plot_series(df_plot)
    
    # Calcular métricas (r, bias, RMSE)
    metrics_text = get_metrics(df_plot)
    
    # Crear subplot
    ax = plt.subplot(2, 2, i)
    
    # Graficar scatter con color según densidad
    sc = ax.scatter(y_obs, y_sim, c=dens, s=15, cmap='viridis')
    
    # Línea 1:1 para referencia
    min_val = min(y_obs.min(), y_sim.min())
    max_val = max(y_obs.max(), y_sim.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=1)
    
    # Etiquetas y título
    ax.set_xlabel('Temperatura Observada (Ta_Avg) °C')
    ax.set_ylabel(f'Temperatura Simulada ({var}) °C')
    ax.set_title(f'Comparación Observado vs {var}')
    
    # Añadir texto con métricas en la esquina superior izquierda
    ax.text(0.05, 0.95, metrics_text, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Colorbar para la densidad
    plt.colorbar(sc, ax=ax, label='Densidad KDE')

plt.tight_layout()
plt.show()


In [ ]:
# Lista de variables simuladas
sim_vars = ['t2m', 't2m_corr_lp', 't2m_corr_lineal', 't2m_corr_xgb']

# Asegurarse de que el índice es datetime
df_t2.index = pd.to_datetime(df_t2.index)

# Agrupar por día y calcular la media
df_daily = df_t2[['Ta_Avg'] + sim_vars].resample('D').mean().dropna()

# Crear figura
plt.figure(figsize=(12, 6))

# Graficar observaciones
plt.plot(df_daily.index, df_daily['Ta_Avg'], label='Observado', color='black', linewidth=1)

# Graficar simulaciones
for var in sim_vars:
    plt.plot(df_daily.index, df_daily[var], label=var, alpha=0.7)

# Personalizar gráfico
plt.title('Serie Temporal Diaria: Temperatura Observada vs Simulaciones')
plt.xlabel('Fecha')
plt.ylabel('Temperatura Media Diaria (°C)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Escalamiento ERA5 para SWin

In [ ]:
# -------------------------------
# 1. Preparación de datos
# -------------------------------

# Definir las variables que se utilizarán en el análisis:
# - 't2m': temperatura del aire a 2 metros (ERA5)
# - 'ssrd': radiación solar superficial descendente (ERA5)
# - 'rh': humedad relativa (ERA5)
# - 'z': altitud del punto ERA5
# - 'ele_aws': altitud de la estación meteorológica
# - 'SWin_Avg': radiación solar media observada en la estación meteorológica
#vars = ['t2m', 'ssrd', 'u10', 'v10', 'Ta_Avg']
vars = ['ssrd', 't2m', 'rh', 'z', 'ele_aws', 'SWin_Avg']

# Crear una copia del DataFrame original que contenga únicamente las variables seleccionadas y eliminar filas con valores nulos
df_era5_aws_nonan = df_era5_aws[vars].copy().dropna()

# Definir las variables predictoras (features) tomadas de ERA5 y altitud de la estación
#X = df_era5_aws_nonan[['t2m', 'ssrd', 'u10', 'v10']]
X = df_era5_aws_nonan[['ssrd', 't2m', 'rh', 'z', 'ele_aws']]

# Definir la variable objetivo: temperatura promedio observada (de la estación)
y = df_era5_aws_nonan['SWin_Avg']

# Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
# La semilla random_state=42 permite obtener los mismos resultados al ejecutar varias veces
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir el conjunto de prueba (features) para revisar su contenido
print(len(X_train), len(X_test))


In [ ]:
print(X_train)

In [ ]:
print(y_train)

In [ ]:
# -------------------------------
# 2. Modelo de regresión lineal
# -------------------------------

# Crear una instancia del modelo de regresión lineal de scikit-learn
modelo_lr = LinearRegression()

# Ajustar (entrenar) el modelo usando los datos de entrenamiento
modelo_lr.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_lr = modelo_lr.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los valores reales
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

# Calcular el coeficiente de determinación R² (explica qué proporción de la varianza se explica por el modelo)
r2_lr = r2_score(y_test, y_pred_lr)

# Imprimir el RMSE con dos decimales, indicando el error promedio de las predicciones
print(f"[Regresión Lineal] RMSE: {rmse_lr:.0f} W/m²")

# Imprimir el R² con dos decimales, indicando la calidad del ajuste del modelo
print(f"[Regresión Lineal] R²: {r2_lr:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['SWin_Avg'])
df_era5_aws['SWin_corr_lineal'] = modelo_lr.predict(X_full)
df_era5_aws

In [ ]:
# -------------------------------
# 4. Modelo XGBoost
# -------------------------------

# Crear un modelo de regresión XGBoost con hiperparámetros definidos:
# - n_estimators=100: número de árboles a construir.
# - learning_rate=0.1: tasa de aprendizaje que controla la contribución de cada árbol.
# - max_depth=4: profundidad máxima de cada árbol (controla la complejidad del modelo).
# - random_state=42: semilla aleatoria para reproducibilidad.
modelo_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)

# Entrenar el modelo con los datos de entrenamiento (X_train, y_train)
modelo_xgb.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_xgb = modelo_xgb.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los datos reales
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

# Calcular el coeficiente de determinación R² (qué tan bien el modelo explica la variabilidad de los datos)
r2_xgb = r2_score(y_test, y_pred_xgb)

# Imprimir el RMSE formateado con dos decimales
print(f"[XGBoost] RMSE: {rmse_xgb:.0f} °W/m²")

# Imprimir el R² formateado con dos decimales
print(f"[XGBoost] R²: {r2_xgb:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['SWin_Avg'])
df_era5_aws['SWin_corr_xgb'] = modelo_xgb.predict(X_full)
df_era5_aws

In [ ]:
vars_SWin = ['SWin_Avg', 'ssrd', 'SWin_corr_lineal', 'SWin_corr_xgb']
df_SWin = df_era5_aws[vars_SWin].resample('1d').mean()
df_SWin

In [ ]:
# Lista de variables simuladas para comparar con observados
sim_vars = ['ssrd', 'SWin_corr_lineal', 'SWin_corr_xgb']

plt.figure(figsize=(18, 5))

for i, var in enumerate(sim_vars, 1):
    # Preparar DataFrame con las columnas observadas y simuladas para la función plot_series y get_metrics
    df_plot = df_SWin[['SWin_Avg', var]].copy()
    
    # Obtener datos para graficar con densidad KDE
    y_obs, y_sim, dens = plot_series(df_plot)
    
    # Calcular métricas (r, bias, RMSE)
    metrics_text = get_metrics(df_plot)
    
    # Crear subplot
    ax = plt.subplot(1, 3, i)
    
    # Graficar scatter con color según densidad
    sc = ax.scatter(y_obs, y_sim, c=dens, s=15, cmap='viridis')
    
    # Línea 1:1 para referencia
    min_val = min(y_obs.min(), y_sim.min())
    max_val = max(y_obs.max(), y_sim.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=1)
    
    # Etiquetas y título
    ax.set_xlabel('SWin Observada (SWin_Avg) W/m²')
    ax.set_ylabel(f'SWin Simulada ({var}) W/m²')
    ax.set_title(f'Comparación Observado vs {var}')
    
    # Añadir texto con métricas en la esquina superior izquierda
    ax.text(0.05, 0.95, metrics_text, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Colorbar para la densidad
    plt.colorbar(sc, ax=ax, label='Densidad KDE')

plt.tight_layout()
plt.show()


### Escalamiento humedad relativa

In [ ]:
# -------------------------------
# 1. Preparación de datos
# -------------------------------

# Definir las variables que se utilizarán en el análisis:
# - 't2m': temperatura del aire a 2 metros (ERA5)
# - 'ssrd': radiación solar superficial descendente (ERA5)
# - 'rh': humedad relativa (ERA5)
# - 'ws2': velocidad del viento (ERA5)
# - 'z': altitud del punto ERA5
# - 'ele_aws': altitud de la estación meteorológica
# - 'Rh_Avg': radiación solar media observada en la estación meteorológica
#vars = ['t2m', 'ssrd', 'u10', 'v10', 'Ta_Avg']
vars = ['rh', 'ssrd', 't2m', 'ws2', 'z', 'ele_aws', 'Rh_Avg']

# Crear una copia del DataFrame original que contenga únicamente las variables seleccionadas y eliminar filas con valores nulos
df_era5_aws_nonan = df_era5_aws[vars].copy().dropna()

# Definir las variables predictoras (features) tomadas de ERA5 y altitud de la estación
#X = df_era5_aws_nonan[['t2m', 'ssrd', 'u10', 'v10']]
X = df_era5_aws_nonan[['rh', 'ssrd', 't2m', 'ws2', 'z', 'ele_aws']]

# Definir la variable objetivo: temperatura promedio observada (de la estación)
y = df_era5_aws_nonan['Rh_Avg']

# Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
# La semilla random_state=42 permite obtener los mismos resultados al ejecutar varias veces
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir el conjunto de prueba (features) para revisar su contenido
print(len(X_train), len(X_test))


In [ ]:
# -------------------------------
# 2. Modelo de regresión lineal
# -------------------------------

# Crear una instancia del modelo de regresión lineal de scikit-learn
modelo_lr = LinearRegression()

# Ajustar (entrenar) el modelo usando los datos de entrenamiento
modelo_lr.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_lr = modelo_lr.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los valores reales
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

# Calcular el coeficiente de determinación R² (explica qué proporción de la varianza se explica por el modelo)
r2_lr = r2_score(y_test, y_pred_lr)

# Imprimir el RMSE con dos decimales, indicando el error promedio de las predicciones
print(f"[Regresión Lineal] RMSE: {rmse_lr:.0f} %")

# Imprimir el R² con dos decimales, indicando la calidad del ajuste del modelo
print(f"[Regresión Lineal] R²: {r2_lr:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['Rh_Avg'])
df_era5_aws['Rh_corr_lineal'] = modelo_lr.predict(X_full)
df_era5_aws

In [ ]:
# -------------------------------
# 4. Modelo XGBoost
# -------------------------------

# Crear un modelo de regresión XGBoost con hiperparámetros definidos:
# - n_estimators=100: número de árboles a construir.
# - learning_rate=0.1: tasa de aprendizaje que controla la contribución de cada árbol.
# - max_depth=4: profundidad máxima de cada árbol (controla la complejidad del modelo).
# - random_state=42: semilla aleatoria para reproducibilidad.
modelo_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)

# Entrenar el modelo con los datos de entrenamiento (X_train, y_train)
modelo_xgb.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_xgb = modelo_xgb.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los datos reales
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

# Calcular el coeficiente de determinación R² (qué tan bien el modelo explica la variabilidad de los datos)
r2_xgb = r2_score(y_test, y_pred_xgb)

# Imprimir el RMSE formateado con dos decimales
print(f"[XGBoost] RMSE: {rmse_xgb:.0f} %")

# Imprimir el R² formateado con dos decimales
print(f"[XGBoost] R²: {r2_xgb:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['Rh_Avg'])
df_era5_aws['Rh_corr_xgb'] = modelo_xgb.predict(X_full)
df_era5_aws

In [ ]:
vars_rh = ['Rh_Avg', 'rh', 'Rh_corr_lineal', 'Rh_corr_xgb']
df_rh = df_era5_aws[vars_rh]#.resample('1d').mean()
df_rh

In [ ]:
# Lista de variables simuladas para comparar con observados
sim_vars = ['rh', 'Rh_corr_lineal', 'Rh_corr_xgb']

plt.figure(figsize=(18, 5))

for i, var in enumerate(sim_vars, 1):
    # Preparar DataFrame con las columnas observadas y simuladas para la función plot_series y get_metrics
    df_plot = df_rh[['Rh_Avg', var]].copy()
    
    # Obtener datos para graficar con densidad KDE
    y_obs, y_sim, dens = plot_series(df_plot)
    
    # Calcular métricas (r, bias, RMSE)
    metrics_text = get_metrics(df_plot)
    
    # Crear subplot
    ax = plt.subplot(1, 3, i)
    
    # Graficar scatter con color según densidad
    sc = ax.scatter(y_obs, y_sim, c=dens, s=15, cmap='viridis')
    
    # Línea 1:1 para referencia
    min_val = min(y_obs.min(), y_sim.min())
    max_val = max(y_obs.max(), y_sim.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=1)
    
    # Etiquetas y título
    ax.set_xlabel('RH Observada (Rh_Avg) %²')
    ax.set_ylabel(f'RH Simulada ({var}) %')
    ax.set_title(f'Comparación Observado vs {var}')
    
    # Añadir texto con métricas en la esquina superior izquierda
    ax.text(0.05, 0.95, metrics_text, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Colorbar para la densidad
    plt.colorbar(sc, ax=ax, label='Densidad KDE')

plt.tight_layout()
plt.show()


### Presion del aire

In [ ]:
# -------------------------------
# 1. Preparación de datos
# -------------------------------

# Definir las variables que se utilizarán en el análisis:
# - 't2m': temperatura del aire a 2 metros (ERA5)
# - 'ssrd': radiación solar superficial descendente (ERA5)
# - 'rh': humedad relativa (ERA5)
# - 'ws2': velocidad del viento (ERA5)
# - 'z': altitud del punto ERA5
# - 'ele_aws': altitud de la estación meteorológica
# - 'Rh_Avg': radiación solar media observada en la estación meteorológica
#vars = ['t2m', 'ssrd', 'u10', 'v10', 'Ta_Avg']
vars = ['sp', 't2m', 'ssrd', 'z', 'ele_aws', 'pressure']

# Crear una copia del DataFrame original que contenga únicamente las variables seleccionadas y eliminar filas con valores nulos
df_era5_aws_nonan = df_era5_aws[vars].copy().dropna()

# Definir las variables predictoras (features) tomadas de ERA5 y altitud de la estación
#X = df_era5_aws_nonan[['t2m', 'ssrd', 'u10', 'v10']]
X = df_era5_aws_nonan[['sp', 't2m', 'ssrd', 'z', 'ele_aws']]

# Definir la variable objetivo: temperatura promedio observada (de la estación)
y = df_era5_aws_nonan['pressure']

# Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
# La semilla random_state=42 permite obtener los mismos resultados al ejecutar varias veces
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir el conjunto de prueba (features) para revisar su contenido
print(len(X_train), len(X_test))


In [ ]:
# -------------------------------
# 2. Modelo de regresión lineal
# -------------------------------

# Crear una instancia del modelo de regresión lineal de scikit-learn
modelo_lr = LinearRegression()

# Ajustar (entrenar) el modelo usando los datos de entrenamiento
modelo_lr.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_lr = modelo_lr.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los valores reales
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

# Calcular el coeficiente de determinación R² (explica qué proporción de la varianza se explica por el modelo)
r2_lr = r2_score(y_test, y_pred_lr)

# Imprimir el RMSE con dos decimales, indicando el error promedio de las predicciones
print(f"[Regresión Lineal] RMSE: {rmse_lr:.2f} hPa")

# Imprimir el R² con dos decimales, indicando la calidad del ajuste del modelo
print(f"[Regresión Lineal] R²: {r2_lr:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['pressure'])
df_era5_aws['sp_corr_lineal'] = modelo_lr.predict(X_full)
df_era5_aws

In [ ]:
# -------------------------------
# 4. Modelo XGBoost
# -------------------------------

# Crear un modelo de regresión XGBoost con hiperparámetros definidos:
# - n_estimators=100: número de árboles a construir.
# - learning_rate=0.1: tasa de aprendizaje que controla la contribución de cada árbol.
# - max_depth=4: profundidad máxima de cada árbol (controla la complejidad del modelo).
# - random_state=42: semilla aleatoria para reproducibilidad.
modelo_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)

# Entrenar el modelo con los datos de entrenamiento (X_train, y_train)
modelo_xgb.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_xgb = modelo_xgb.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los datos reales
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

# Calcular el coeficiente de determinación R² (qué tan bien el modelo explica la variabilidad de los datos)
r2_xgb = r2_score(y_test, y_pred_xgb)

# Imprimir el RMSE formateado con dos decimales
print(f"[XGBoost] RMSE: {rmse_xgb:.2f} hPa")

# Imprimir el R² formateado con dos decimales
print(f"[XGBoost] R²: {r2_xgb:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['pressure'])
df_era5_aws['sp_corr_xgb'] = modelo_xgb.predict(X_full)
df_era5_aws

In [ ]:
vars_sp = ['pressure', 'sp', 'sp_corr_lineal', 'sp_corr_xgb']
df_sp = df_era5_aws[vars_sp]#.resample('1d').mean()
df_sp

In [ ]:
# Lista de variables simuladas para comparar con observados
sim_vars = ['sp', 'sp_corr_lineal', 'sp_corr_xgb']

plt.figure(figsize=(18, 5))

for i, var in enumerate(sim_vars, 1):
    # Preparar DataFrame con las columnas observadas y simuladas para la función plot_series y get_metrics
    df_plot = df_sp[['pressure', var]].copy()
    
    # Obtener datos para graficar con densidad KDE
    y_obs, y_sim, dens = plot_series(df_plot)
    
    # Calcular métricas (r, bias, RMSE)
    metrics_text = get_metrics(df_plot)
    
    # Crear subplot
    ax = plt.subplot(1, 3, i)
    
    # Graficar scatter con color según densidad
    sc = ax.scatter(y_obs, y_sim, c=dens, s=15, cmap='viridis')
    
    # Línea 1:1 para referencia
    min_val = min(y_obs.min(), y_sim.min())
    max_val = max(y_obs.max(), y_sim.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=1)
    
    # Etiquetas y título
    ax.set_xlabel('SP Observada (Rh_Avg) hPa')
    ax.set_ylabel(f'SP Simulada ({var}) hPa')
    ax.set_title(f'Comparación Observado vs {var}')
    
    # Añadir texto con métricas en la esquina superior izquierda
    ax.text(0.05, 0.95, metrics_text, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Colorbar para la densidad
    plt.colorbar(sc, ax=ax, label='Densidad KDE')

plt.tight_layout()
plt.show()


### Velocidad del viento

In [ ]:
# -------------------------------
# 1. Preparación de datos
# -------------------------------

# Definir las variables que se utilizarán en el análisis:
# - 't2m': temperatura del aire a 2 metros (ERA5)
# - 'ssrd': radiación solar superficial descendente (ERA5)
# - 'rh': humedad relativa (ERA5)
# - 'ws2': velocidad del viento (ERA5)
# - 'z': altitud del punto ERA5
# - 'ele_aws': altitud de la estación meteorológica
# - 'Rh_Avg': radiación solar media observada en la estación meteorológica
#vars = ['t2m', 'ssrd', 'u10', 'v10', 'Ta_Avg']
vars = ['ws2', 'u10', 'v10', 't2m', 'ssrd', 'sp', 'z', 'ele_aws', 'WiSp']

# Crear una copia del DataFrame original que contenga únicamente las variables seleccionadas y eliminar filas con valores nulos
df_era5_aws_nonan = df_era5_aws[vars].copy().dropna()

# Definir las variables predictoras (features) tomadas de ERA5 y altitud de la estación
#X = df_era5_aws_nonan[['t2m', 'ssrd', 'u10', 'v10']]
X = df_era5_aws_nonan[['ws2', 'u10', 'v10', 't2m', 'ssrd', 'sp', 'z', 'ele_aws']]

# Definir la variable objetivo: temperatura promedio observada (de la estación)
y = df_era5_aws_nonan['WiSp']

# Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
# La semilla random_state=42 permite obtener los mismos resultados al ejecutar varias veces
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir el conjunto de prueba (features) para revisar su contenido
print(len(X_train), len(X_test))


In [ ]:
# -------------------------------
# 2. Modelo de regresión lineal
# -------------------------------

# Crear una instancia del modelo de regresión lineal de scikit-learn
modelo_lr = LinearRegression()

# Ajustar (entrenar) el modelo usando los datos de entrenamiento
modelo_lr.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_lr = modelo_lr.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los valores reales
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

# Calcular el coeficiente de determinación R² (explica qué proporción de la varianza se explica por el modelo)
r2_lr = r2_score(y_test, y_pred_lr)

# Imprimir el RMSE con dos decimales, indicando el error promedio de las predicciones
print(f"[Regresión Lineal] RMSE: {rmse_lr:.2f} m/s")

# Imprimir el R² con dos decimales, indicando la calidad del ajuste del modelo
print(f"[Regresión Lineal] R²: {r2_lr:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['WiSp'])
df_era5_aws['ws2_corr_lineal'] = modelo_lr.predict(X_full)
df_era5_aws

In [ ]:
# -------------------------------
# 4. Modelo XGBoost
# -------------------------------

# Crear un modelo de regresión XGBoost con hiperparámetros definidos:
# - n_estimators=100: número de árboles a construir.
# - learning_rate=0.1: tasa de aprendizaje que controla la contribución de cada árbol.
# - max_depth=4: profundidad máxima de cada árbol (controla la complejidad del modelo).
# - random_state=42: semilla aleatoria para reproducibilidad.
modelo_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)

# Entrenar el modelo con los datos de entrenamiento (X_train, y_train)
modelo_xgb.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_xgb = modelo_xgb.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los datos reales
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

# Calcular el coeficiente de determinación R² (qué tan bien el modelo explica la variabilidad de los datos)
r2_xgb = r2_score(y_test, y_pred_xgb)

# Imprimir el RMSE formateado con dos decimales
print(f"[XGBoost] RMSE: {rmse_xgb:.2f} m/s")

# Imprimir el R² formateado con dos decimales
print(f"[XGBoost] R²: {r2_xgb:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['WiSp'])
df_era5_aws['ws2_corr_xgb'] = modelo_xgb.predict(X_full)
df_era5_aws

In [ ]:
vars_ws2 = ['WiSp', 'ws2', 'ws2_corr_lineal', 'ws2_corr_xgb']
df_ws2 = df_era5_aws[vars_ws2]#.resample('1d').mean()
df_ws2

In [ ]:
# Lista de variables simuladas para comparar con observados
sim_vars = ['ws2', 'ws2_corr_lineal', 'ws2_corr_xgb']

plt.figure(figsize=(18, 5))

for i, var in enumerate(sim_vars, 1):
    # Preparar DataFrame con las columnas observadas y simuladas para la función plot_series y get_metrics
    df_plot = df_ws2[['WiSp', var]].copy()
    
    # Obtener datos para graficar con densidad KDE
    y_obs, y_sim, dens = plot_series(df_plot)
    
    # Calcular métricas (r, bias, RMSE)
    metrics_text = get_metrics(df_plot)
    
    # Crear subplot
    ax = plt.subplot(1, 3, i)
    
    # Graficar scatter con color según densidad
    sc = ax.scatter(y_obs, y_sim, c=dens, s=15, cmap='viridis')
    
    # Línea 1:1 para referencia
    min_val = min(y_obs.min(), y_sim.min())
    max_val = max(y_obs.max(), y_sim.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=1)
    
    # Etiquetas y título
    ax.set_xlabel('WS2 Observada (WiSp) m/s')
    ax.set_ylabel(f'WS2 Simulada ({var}) m/s')
    ax.set_title(f'Comparación Observado vs {var}')
    
    # Añadir texto con métricas en la esquina superior izquierda
    ax.text(0.05, 0.95, metrics_text, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Colorbar para la densidad
    plt.colorbar(sc, ax=ax, label='Densidad KDE')

plt.tight_layout()
plt.show()


In [ ]:
# Lista de variables simuladas
sim_vars = ['ws2', 'ws2_corr_lineal', 'ws2_corr_xgb']

# Asegurarse de que el índice es datetime
df_ws2.index = pd.to_datetime(df_ws2.index)

# Agrupar por día y calcular la media
df_daily = df_ws2[['WiSp'] + sim_vars].resample('D').mean().dropna()

# Crear figura
plt.figure(figsize=(12, 6))

# Graficar observaciones
plt.plot(df_daily.index, df_daily['WiSp'], label='Observado', color='black', linewidth=1)

# Graficar simulaciones
for var in sim_vars:
    plt.plot(df_daily.index, df_daily[var], label=var, alpha=0.7)

# Personalizar gráfico
plt.title('Serie Temporal Diaria: Velocidad del Viento Observada vs Simulaciones')
plt.xlabel('Fecha')
plt.ylabel('Velocidad de viento (m/s)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


#### Correcion incluyendo un analisis climatico

In [ ]:
# -------------------------------
# 1. Preparación de datos
# -------------------------------

# Definir las variables que se utilizarán en el análisis:
# - 't2m': temperatura del aire a 2 metros (ERA5)
# - 'ssrd': radiación solar superficial descendente (ERA5)
# - 'rh': humedad relativa (ERA5)
# - 'ws2': velocidad del viento (ERA5)
# - 'z': altitud del punto ERA5
# - 'ele_aws': altitud de la estación meteorológica
# - 'Rh_Avg': radiación solar media observada en la estación meteorológica
#vars = ['t2m', 'ssrd', 'u10', 'v10', 'Ta_Avg']
vars = ['ws2', 'u10', 'v10', 't2m', 'ssrd', 'sp', 'z', 'ele_aws', 'WiSp']

# Crear una copia del DataFrame original que contenga únicamente las variables seleccionadas y eliminar filas con valores nulos
df_era5_aws = df_era5_aws[vars].copy()#.dropna()
df_era5_aws


In [ ]:
df_era5_aws['day_of_year'] = df_era5_aws.index.dayofyear
df_era5_aws

In [ ]:
df_era5_aws['hour_group'] = (df_era5_aws.index.hour // 3) * 3
df_era5_aws

In [ ]:
clim_cols = ['day_of_year', 'hour_group']
hourly_clim = df_era5_aws.groupby(clim_cols)['WiSp'].agg(['mean']).rename(columns={'mean':'WiSp_clim'})
hourly_clim

In [ ]:
hourly_clim['WiSp_clim'].plot()

In [ ]:
df_era5_aws = df_era5_aws.join(hourly_clim, on=clim_cols)
df_era5_aws

In [ ]:
df_era5_aws = df_era5_aws.drop(columns=['day_of_year', 'hour_group'])
df_era5_aws

In [ ]:
df_era5_aws_nonan = df_era5_aws.copy().dropna()
# Definir las variables predictoras (features) tomadas de ERA5 y altitud de la estación
#X = df_era5_aws_nonan[['t2m', 'ssrd', 'u10', 'v10']]
X = df_era5_aws_nonan[['ws2', 'WiSp_clim','u10', 'v10', 't2m', 'ssrd', 'sp', 'z', 'ele_aws']]

# Definir la variable objetivo: temperatura promedio observada (de la estación)
y = df_era5_aws_nonan['WiSp']

# Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
# La semilla random_state=42 permite obtener los mismos resultados al ejecutar varias veces
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir el conjunto de prueba (features) para revisar su contenido
print(len(X_train), len(X_test))


In [ ]:
# -------------------------------
# 2. Modelo de regresión lineal
# -------------------------------

# Crear una instancia del modelo de regresión lineal de scikit-learn
modelo_lr = LinearRegression()

# Ajustar (entrenar) el modelo usando los datos de entrenamiento
modelo_lr.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_lr = modelo_lr.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los valores reales
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

# Calcular el coeficiente de determinación R² (explica qué proporción de la varianza se explica por el modelo)
r2_lr = r2_score(y_test, y_pred_lr)

# Imprimir el RMSE con dos decimales, indicando el error promedio de las predicciones
print(f"[Regresión Lineal] RMSE: {rmse_lr:.2f} m/s")

# Imprimir el R² con dos decimales, indicando la calidad del ajuste del modelo
print(f"[Regresión Lineal] R²: {r2_lr:.2f}")


In [ ]:
vars_clim = ['ws2', 'WiSp_clim','u10', 'v10', 't2m', 'ssrd', 'sp', 'z', 'ele_aws']

In [ ]:
X_full = df_era5_aws[vars_clim]
df_era5_aws['ws2_clim_corr_lineal'] = modelo_lr.predict(X_full)
df_era5_aws

In [ ]:
# -------------------------------
# 4. Modelo XGBoost
# -------------------------------

# Crear un modelo de regresión XGBoost con hiperparámetros definidos:
# - n_estimators=100: número de árboles a construir.
# - learning_rate=0.1: tasa de aprendizaje que controla la contribución de cada árbol.
# - max_depth=4: profundidad máxima de cada árbol (controla la complejidad del modelo).
# - random_state=42: semilla aleatoria para reproducibilidad.
modelo_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)

# Entrenar el modelo con los datos de entrenamiento (X_train, y_train)
modelo_xgb.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_xgb = modelo_xgb.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los datos reales
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

# Calcular el coeficiente de determinación R² (qué tan bien el modelo explica la variabilidad de los datos)
r2_xgb = r2_score(y_test, y_pred_xgb)

# Imprimir el RMSE formateado con dos decimales
print(f"[XGBoost] RMSE: {rmse_xgb:.2f} m/s")

# Imprimir el R² formateado con dos decimales
print(f"[XGBoost] R²: {r2_xgb:.2f}")


In [ ]:
X_full = df_era5_aws[vars_clim]
df_era5_aws['ws2_clim_corr_xgb'] = modelo_xgb.predict(X_full)
df_era5_aws

In [ ]:
vars_ws2 = ['WiSp', 'ws2', 'ws2_clim_corr_lineal', 'ws2_clim_corr_xgb']
df_ws2 = df_era5_aws[vars_ws2]#.resample('1d').mean()
df_ws2

In [ ]:
# Lista de variables simuladas para comparar con observados
sim_vars = ['ws2', 'ws2_clim_corr_lineal', 'ws2_clim_corr_xgb']

plt.figure(figsize=(18, 5))

for i, var in enumerate(sim_vars, 1):
    # Preparar DataFrame con las columnas observadas y simuladas para la función plot_series y get_metrics
    df_plot = df_ws2[['WiSp', var]].copy()
    
    # Obtener datos para graficar con densidad KDE
    y_obs, y_sim, dens = plot_series(df_plot)
    
    # Calcular métricas (r, bias, RMSE)
    metrics_text = get_metrics(df_plot)
    
    # Crear subplot
    ax = plt.subplot(1, 3, i)
    
    # Graficar scatter con color según densidad
    sc = ax.scatter(y_obs, y_sim, c=dens, s=15, cmap='viridis')
    
    # Línea 1:1 para referencia
    min_val = min(y_obs.min(), y_sim.min())
    max_val = max(y_obs.max(), y_sim.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=1)
    
    # Etiquetas y título
    ax.set_xlabel('WS2 Observada (WiSp) m/s')
    ax.set_ylabel(f'WS2 Simulada ({var}) m/s')
    ax.set_title(f'Comparación Observado vs {var}')
    
    # Añadir texto con métricas en la esquina superior izquierda
    ax.text(0.05, 0.95, metrics_text, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Colorbar para la densidad
    plt.colorbar(sc, ax=ax, label='Densidad KDE')

plt.tight_layout()
plt.show()


In [ ]:
# Lista de variables simuladas
sim_vars = ['ws2', 'ws2_clim_corr_lineal', 'ws2_clim_corr_xgb']

# Asegurarse de que el índice es datetime
df_ws2.index = pd.to_datetime(df_ws2.index)

# Agrupar por día y calcular la media
df_daily = df_ws2[['WiSp'] + sim_vars].resample('D').mean().dropna()

# Crear figura
plt.figure(figsize=(12, 6))

# Graficar observaciones
plt.plot(df_daily.index, df_daily['WiSp'], label='Observado', color='black', linewidth=1)

# Graficar simulaciones
for var in sim_vars:
    plt.plot(df_daily.index, df_daily[var], label=var, alpha=0.7)

# Personalizar gráfico
plt.title('Serie Temporal Diaria: Velocidad del Viento Observada vs Simulaciones')
plt.xlabel('Fecha')
plt.ylabel('Velocidad de viento (m/s)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### Precipitacion

In [ ]:
# -------------------------------
# 1. Preparación de datos
# -------------------------------

# Definir las variables que se utilizarán en el análisis:
# - 't2m': temperatura del aire a 2 metros (ERA5)
# - 'ssrd': radiación solar superficial descendente (ERA5)
# - 'rh': humedad relativa (ERA5)
# - 'ws2': velocidad del viento (ERA5)
# - 'z': altitud del punto ERA5
# - 'ele_aws': altitud de la estación meteorológica
# - 'Rh_Avg': radiación solar media observada en la estación meteorológica
#vars = ['t2m', 'ssrd', 'u10', 'v10', 'Ta_Avg']
vars = ['tp', 't2m', 'ssrd', 'rh',  'z', 'ele_aws', 'precip_Tot']

# Crear una copia del DataFrame original que contenga únicamente las variables seleccionadas y eliminar filas con valores nulos
df_era5_aws_nonan = df_era5_aws[vars].copy().dropna()

# Definir las variables predictoras (features) tomadas de ERA5 y altitud de la estación
#X = df_era5_aws_nonan[['t2m', 'ssrd', 'u10', 'v10']]
X = df_era5_aws_nonan[['tp', 't2m', 'ssrd', 'rh',  'z', 'ele_aws']]

# Definir la variable objetivo: temperatura promedio observada (de la estación)
y = df_era5_aws_nonan['precip_Tot']

# Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
# La semilla random_state=42 permite obtener los mismos resultados al ejecutar varias veces
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir el conjunto de prueba (features) para revisar su contenido
print(len(X_train), len(X_test))

In [ ]:
# -------------------------------
# 2. Modelo de regresión lineal
# -------------------------------

# Crear una instancia del modelo de regresión lineal de scikit-learn
modelo_lr = LinearRegression()

# Ajustar (entrenar) el modelo usando los datos de entrenamiento
modelo_lr.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_lr = modelo_lr.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los valores reales
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

# Calcular el coeficiente de determinación R² (explica qué proporción de la varianza se explica por el modelo)
r2_lr = r2_score(y_test, y_pred_lr)

# Imprimir el RMSE con dos decimales, indicando el error promedio de las predicciones
print(f"[Regresión Lineal] RMSE: {rmse_lr:.2f} mm/hr")

# Imprimir el R² con dos decimales, indicando la calidad del ajuste del modelo
print(f"[Regresión Lineal] R²: {r2_lr:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['precip_Tot'])
df_era5_aws['tp_corr_lineal'] = modelo_lr.predict(X_full)
df_era5_aws

In [ ]:
# -------------------------------
# 4. Modelo XGBoost
# -------------------------------

# Crear un modelo de regresión XGBoost con hiperparámetros definidos:
# - n_estimators=100: número de árboles a construir.
# - learning_rate=0.1: tasa de aprendizaje que controla la contribución de cada árbol.
# - max_depth=4: profundidad máxima de cada árbol (controla la complejidad del modelo).
# - random_state=42: semilla aleatoria para reproducibilidad.
modelo_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)

# Entrenar el modelo con los datos de entrenamiento (X_train, y_train)
modelo_xgb.fit(X_train, y_train)

# Usar el modelo entrenado para hacer predicciones sobre los datos de prueba
y_pred_xgb = modelo_xgb.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE) entre las predicciones y los datos reales
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

# Calcular el coeficiente de determinación R² (qué tan bien el modelo explica la variabilidad de los datos)
r2_xgb = r2_score(y_test, y_pred_xgb)

# Imprimir el RMSE formateado con dos decimales
print(f"[XGBoost] RMSE: {rmse_xgb:.2f} mm")

# Imprimir el R² formateado con dos decimales
print(f"[XGBoost] R²: {r2_xgb:.2f}")


In [ ]:
X_full = df_era5_aws[vars].drop(columns=['precip_Tot'])
df_era5_aws['tp_corr_xgb'] = modelo_xgb.predict(X_full)
df_era5_aws

In [ ]:
vars_tp = ['precip_Tot', 'tp', 'tp_corr_lineal', 'tp_corr_xgb']
df_tp = df_era5_aws[vars_tp].resample('1d').mean()
df_tp

In [ ]:
# Lista de variables simuladas para comparar con observados
sim_vars = ['tp', 'tp_corr_lineal', 'tp_corr_xgb']

plt.figure(figsize=(18, 5))

for i, var in enumerate(sim_vars, 1):
    # Preparar DataFrame con las columnas observadas y simuladas para la función plot_series y get_metrics
    df_plot = df_tp[['precip_Tot', var]].copy()
    
    # Obtener datos para graficar con densidad KDE
    y_obs, y_sim, dens = plot_series(df_plot)
    
    # Calcular métricas (r, bias, RMSE)
    metrics_text = get_metrics(df_plot)
    
    # Crear subplot
    ax = plt.subplot(1, 3, i)
    
    # Graficar scatter con color según densidad
    sc = ax.scatter(y_obs, y_sim, c=dens, s=15, cmap='viridis')
    
    # Línea 1:1 para referencia
    min_val = min(y_obs.min(), y_sim.min())
    max_val = max(y_obs.max(), y_sim.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=1)
    
    # Etiquetas y título
    ax.set_xlabel('TP Observada (precip_Tot) mm')
    ax.set_ylabel(f'TP Simulada ({var}) mm')
    ax.set_title(f'Comparación Observado vs {var}')
    
    # Añadir texto con métricas en la esquina superior izquierda
    ax.text(0.05, 0.95, metrics_text, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Colorbar para la densidad
    plt.colorbar(sc, ax=ax, label='Densidad KDE')

plt.tight_layout()
plt.show()


In [ ]:
# Lista de variables simuladas
sim_vars = ['tp', 'tp_corr_lineal', 'tp_corr_xgb']

# Asegurarse de que el índice es datetime
df_ws2.index = pd.to_datetime(df_ws2.index)

# Agrupar por día y calcular la media
df_daily = df_tp[['precip_Tot'] + sim_vars].resample('D').mean().dropna()

# Crear figura
plt.figure(figsize=(12, 6))

# Graficar observaciones
plt.plot(df_daily.index, df_daily['precip_Tot'], label='Observado', color='black', linewidth=1)

# Graficar simulaciones
for var in sim_vars:
    plt.plot(df_daily.index, df_daily[var], label=var, alpha=0.7)

# Personalizar gráfico
plt.title('Serie Temporal Diaria: Precipitacion Observada vs Simulaciones')
plt.xlabel('Fecha')
plt.ylabel('Precipitacion (mm)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
